Here the sceanrio is where 1 data scientist member is working

MLflow setup:

1. Tracking server setup: Local
2. Backend Store: SQLite DB
3. Artifact Store: Local Filesystem

Here first start the tracking server with sqlite backend

''' mlflow server --backend-store-uri sqlite:///backend.db '''

In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [4]:
print(f"tracking_uri: {mlflow.get_tracking_uri()}")

tracking_uri: http://127.0.0.1:5000


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1752937824110, experiment_id='0', last_update_time=1752937824110, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/19 15:29:54 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/07/19 15:29:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/19 15:29:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/1/0f56cfa358c543d1bd5a5dc3fb5a3d95/artifacts'
🏃 View run abundant-bug-386 at: http://127.0.0.1:5000/#/experiments/1/runs/0f56cfa358c543d1bd5a5dc3fb5a3d95
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [7]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1752938994248, experiment_id='1', last_update_time=1752938994248, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1752937824110, experiment_id='0', last_update_time=1752937824110, lifecycle_stage='active', name='Default', tags={}>]

Interacting with model registry

In [10]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [11]:
client.search_registered_models()

[]

In [12]:
#Register a model

run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name = "iris-classification-model"
    )

Successfully registered model 'iris-classification-model'.
2025/07/19 15:45:38 WARNING mlflow.tracking._model_registry.fluent: Run with id 0f56cfa358c543d1bd5a5dc3fb5a3d95 has no artifacts at artifact path 'models', registering model based on models:/m-18d53ccc2fbc48da9e2f96144b73644e instead
2025/07/19 15:45:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classification-model, version 1
Created version '1' of model 'iris-classification-model'.


<ModelVersion: aliases=[], creation_timestamp=1752939938893, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1752939938893, metrics=None, model_id=None, name='iris-classification-model', params=None, run_id='0f56cfa358c543d1bd5a5dc3fb5a3d95', run_link='', source='models:/m-18d53ccc2fbc48da9e2f96144b73644e', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [13]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1752939938742, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', description='', last_updated_timestamp=1752939938893, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1752939938893, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1752939938893, metrics=None, model_id=None, name='iris-classification-model', params=None, run_id='0f56cfa358c543d1bd5a5dc3fb5a3d95', run_link='', source='models:/m-18d53ccc2fbc48da9e2f96144b73644e', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='iris-classification-model', tags={}>]